In [16]:
# Dependencies
from bs4 import BeautifulSoup
import requests
from splinter import Browser
from bs4 import BeautifulSoup
import time
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import pymongo

In [17]:
# Add splinter functions
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 102.0.5005
[WDM] - Get LATEST chromedriver version for 102.0.5005 google-chrome
[WDM] - Driver [C:\Users\Nino\.wdm\drivers\chromedriver\win32\102.0.5005.61\chromedriver.exe] found in cache


In [20]:
# Step 1a - Scraping - NASA Mars News
# Visit Page and add wait time to ensure page loads correctly
url1 = 'https://redplanetscience.com/'
browser.visit(url1)
time.sleep(2)

# Convert the browser HTML to a soup object
soup = BeautifulSoup(browser.html, 'html.parser')

# Drill down into the html tags you want to scrape
results = soup.find('div', class_="list_text").text

# # scrape the article header 
news_title = soup.find('div', class_="content_title").text
news_p = soup.find('div', class_="article_teaser_body").text

print(news_title)
print(news_p)

InSight's 'Mole' Team Peers into the Pit
Efforts to save the heat probe continue.


In [25]:
# Step 1b - JPL Mars Space Images - Featured Image
# Visit the url for the Featured Space Image page [here](https://spaceimages-mars.com).
url2 = 'https://spaceimages-mars.com/'
browser.visit(url2)
time.sleep(2)

# Click the button to view full image
fullimageclick = browser.find_by_tag("button")[1].click()

soup = BeautifulSoup(browser.html, 'html.parser')

# Use splinter to navigate the site and find the image url for the current Featured Mars Image and assign the url string to a variable called `featured_image_url`.
featureimage = soup.find('img', class_="fancybox-image").get("src")

# Make sure to find the image url to the full size `.jpg` image.
featured_image_url = "https://spaceimages-mars.com/" + featureimage

# Print result - Make sure to save a complete url string for this image.
print(featured_image_url)

https://spaceimages-mars.com/image/featured/mars2.jpg


In [22]:
# Step 1c - Mars Facts
# Visit the Mars Facts webpage [here](https://galaxyfacts-mars.com) and use Pandas to scrape the table containing facts about the planet including Diameter, Mass, etc.

url3 = "https://galaxyfacts-mars.com"
table = pd.read_html(url3)

# Find and import the new table from webpage
mars_df = table[0]
mars_df

# Cleanup column names
cols = list(mars_df.columns)
cols[0] = "Descriptor"
cols[1] = "Mars"
cols[2] = "Earth"

# Update the Dataframe and move item column to index
mars_df.columns = cols
mars_df = mars_df.iloc[1:]
mars_df.set_index('Descriptor', inplace=True)

# Convert table to HTML and remove additional lines appear in the HTML
html_table = mars_df.to_html()
html_table = html_table.replace('\n', '')

# Print result
print(html_table)

<table border="1" class="dataframe">  <thead>    <tr style="text-align: right;">      <th></th>      <th>Mars</th>      <th>Earth</th>    </tr>    <tr>      <th>Descriptor</th>      <th></th>      <th></th>    </tr>  </thead>  <tbody>    <tr>      <th>Diameter:</th>      <td>6,779 km</td>      <td>12,742 km</td>    </tr>    <tr>      <th>Mass:</th>      <td>6.39 × 10^23 kg</td>      <td>5.97 × 10^24 kg</td>    </tr>    <tr>      <th>Moons:</th>      <td>2</td>      <td>1</td>    </tr>    <tr>      <th>Distance from Sun:</th>      <td>227,943,824 km</td>      <td>149,598,262 km</td>    </tr>    <tr>      <th>Length of Year:</th>      <td>687 Earth days</td>      <td>365.24 days</td>    </tr>    <tr>      <th>Temperature:</th>      <td>-87 to -5 °C</td>      <td>-88 to 58°C</td>    </tr>  </tbody></table>


In [27]:
# Step 1d - Mars Hemispheres

# Visit the url for the Featured Space Image page [here](https://spaceimages-mars.com).
url4 = 'https://marshemispheres.com/'
browser.visit(url4)
time.sleep(2)

# Create a master  list to store the overall data 
hemisphere_image_urls = []

# Add a new variable for the number of products there are
hemisphere_count = browser.find_by_css("a.product-item img")

# Create a loop which occurs the number of times of "hemisphere_count" occurs 
for count in range(len(hemisphere_count)):
    hemisphere = {}

    # Search for the image, then click the image to get the link to the "Sample" image
    browser.find_by_css("a.product-item img")[count].click()

    # Add dictionary labels to the two variables
    title_dict = browser.find_by_css("h2.title").text
    image_dict = browser.find_by_text("Sample").first

    # Join the two lists together then append to Hemisphere list
    hemisphere["title"] = title_dict
    hemisphere["img_url"] = image_dict["href"]
    hemisphere_image_urls.append(hemisphere)

    # Send the brower back, to repeat the process
    browser.back()

# Print result
print(hemisphere_image_urls)

[{'title': 'Cerberus Hemisphere Enhanced',
  'img_url': 'https://marshemispheres.com/images/full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'img_url': 'https://marshemispheres.com/images/schiaparelli_enhanced-full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'img_url': 'https://marshemispheres.com/images/syrtis_major_enhanced-full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'img_url': 'https://marshemispheres.com/images/valles_marineris_enhanced-full.jpg'}]

In [ ]:
browser.quit()